In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import pickle
import os

# The curve generator function
def generate_qpcr_curve(cycles=40, baseline=1000, plateau=150000, efficiency=1.98, ct_shift=0, noise_level=60, false_positive=False, inhibited=False):
    cycle = np.arange(1, cycles + 1)
    fluorescence = baseline + (plateau - baseline) / (1 + np.exp(-efficiency * (cycle - (20 + ct_shift))))
    fluorescence += np.random.normal(0, noise_level, cycles)
    
    if false_positive:
        fluorescence[:15] += np.linspace(0, 5000, 15) + np.random.normal(0, 200, 15)
        plateau = 20000  # weaker plateau
    
    if inhibited:
        fluorescence[30:] *= np.linspace(1, 0.6, 10)
    
    fluorescence = np.maximum(fluorescence, baseline / 2)
    return fluorescence  # only return fluorescence array

# Generate training data
num_samples = 2000
curves = []
labels = []

np.random.seed(123)

for _ in range(num_samples):
    rnd = np.random.rand()
    if rnd < 0.3:  # Valid Positive
        ct = np.random.uniform(12, 35)
        fluo = generate_qpcr_curve(ct_shift=ct-20)
        curves.append(fluo)
        labels.append("Valid Positive")
    elif rnd < 0.5:  # True Negative
        fluo = generate_qpcr_curve(efficiency=1.0, plateau=5000, noise_level=100)
        curves.append(fluo)
        labels.append("True Negative")
    elif rnd < 0.8:  # False Positive
        fluo = generate_qpcr_curve(false_positive=True, efficiency=1.25, plateau=20000, noise_level=300)
        curves.append(fluo)
        labels.append("False Positive")
    else:  # Inhibited
        fluo = generate_qpcr_curve(ct_shift=np.random.uniform(-12,-2), inhibited=True, plateau=80000)
        curves.append(fluo)
        labels.append("Inhibited")

# Normalize (baseline subtract only — no height scaling)
data = np.array(curves)
baseline = data[:, :15].mean(axis=1, keepdims=True)
data_normalized = data - baseline

X = torch.tensor(data_normalized, dtype=torch.float32).unsqueeze(1)  # [samples, 1, 40]
le = LabelEncoder()
y = torch.tensor(le.fit_transform(labels), dtype=torch.long)

print("Classes:", le.classes_)

# Model
class CurveClassifier(nn.Module):
    def __init__(self, num_classes=4):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv1d(1, 16, kernel_size=5, padding=2),
            nn.ReLU(),
            nn.Conv1d(16, 32, kernel_size=5, padding=2),
            nn.ReLU(),
            nn.AdaptiveMaxPool1d(4)
        )
        self.fc = nn.Sequential(
            nn.Linear(32 * 4, 64),
            nn.ReLU(),
            nn.Linear(64, num_classes)
        )
    
    def forward(self, x):
        x = self.conv(x)
        x = x.flatten(1)
        return self.fc(x)

model = CurveClassifier().to('cuda')

# Training
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)

train_dataset = torch.utils.data.TensorDataset(X_train, y_train)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0005)

for epoch in range(50):
    model.train()
    for batch_x, batch_y in train_loader:
        batch_x, batch_y = batch_x.to('cuda'), batch_y.to('cuda')
        optimizer.zero_grad()
        outputs = model(batch_x)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()
    
    if epoch % 10 == 0:
        print(f"Epoch {epoch}, Loss: {loss.item():.4f}")

print("Training complete!")

# Save the model and classes
torch.save(model.state_dict(), 'qpcr_model.pth')
with open('classes.pkl', 'wb') as f:
    pickle.dump(le.classes_, f)

print("Model saved as qpcr_model.pth")
print("Classes saved as classes.pkl")

Classes: ['False Positive' 'Inhibited' 'True Negative' 'Valid Positive']


/home/shanrajput/jupyterlab/.venv/lib/python3.12/site-packages/torch/cuda/__init__.py:283: UserWarning: 
    Found GPU0 NVIDIA GB10 which is of cuda capability 12.1.
    Minimum and Maximum cuda capability supported by this version of PyTorch is
    (8.0) - (12.0)
    
  warnings.warn(


Epoch 0, Loss: 18.4333
Epoch 10, Loss: 0.0000
Epoch 20, Loss: 0.0000
Epoch 30, Loss: 0.0000
Epoch 40, Loss: 0.0000
Training complete!
Model saved as qpcr_model.pth
Classes saved as classes.pkl
